In [129]:
import numpy as np
import pandas as pd

In [92]:
def generate_request_time_linear(ts_min, ts_max, work_time):
    T_server = [0]
    while True:
        new_t = T_server[-1] + (ts_max - ts_min) * np.random.rand() + ts_min
        if new_t > work_time:
            break
        T_server.append(new_t)
    T_server.pop(0)
    return np.asarray(T_server)

def generate_process_time_linear(tz_min, tz_max, n):
    return (tz_max - tz_min) * np.random.rand(n) + tz_min

def exp_func(lmbd, p):
    return -1 / lmbd * np.log(p)

def generate_request_time_exp(lmbd, work_time):
    T_server = [0]
    while True:
        new_t = T_server[-1] + exp_func(lmbd, np.random.rand())
        if new_t > work_time:
            break
        T_server.append(new_t)
    T_server.pop(0)
    return np.asarray(T_server)

def generate_process_time_exp(lmbd, n):
    return exp_func(lmbd, np.random.rand(n))

def get_f_o_t(cur_t, last_start_s1, last_end_s1, last_start_s2, last_end_s2):
    last_t = max(last_start_s1, last_start_s2)
    two = max(0, (min(last_end_s1, last_end_s2) - last_t))
    free = max(0, cur_t - max(last_end_s1, last_end_s2))
    one = cur_t - last_t - two - free
    return (free, one, two)

def simulate(Ts, Tz, work_time):
    last_start_s1 = 0
    last_end_s1 = 0
    last_start_s2 = 0
    last_end_s2 = 0
    processed_signals = 0

    free_time = 0
    one_work_time = 0
    two_work_time = 0

    for i in range(len(Ts)):
        if Ts[i] + Tz[i] <= work_time:
            if last_end_s1 <= Ts[i]:
                free, one, two = get_f_o_t(Ts[i], last_start_s1, last_end_s1, last_start_s2, last_end_s2)
                free_time += free
                one_work_time += one
                two_work_time += two

                last_start_s1, last_end_s1 = Ts[i], Ts[i] + Tz[i]
                processed_signals += 1
            elif last_end_s2 <= Ts[i]:
                free, one, two = get_f_o_t(Ts[i], last_start_s1, last_end_s1, last_start_s2, last_end_s2)
                free_time += free
                one_work_time += one
                two_work_time += two

                last_start_s2, last_end_s2 = Ts[i], Ts[i] + Tz[i]
                processed_signals += 1
    
    free, one, two = get_f_o_t(work_time, last_start_s1, last_end_s1, last_start_s2, last_end_s2)
    free_time += free
    one_work_time += one
    two_work_time += two

    return (processed_signals, (free_time, one_work_time, two_work_time))

In [169]:
work_time = 1 * 60 * 60

# linear
tz_min = 1 / 2
tz_max = 5 / 6

ts_min = 1
ts_max = 5

lmbd_z_lin = 2 / (tz_max + tz_min)
lmbd_s_lin = 2 / (ts_max + ts_min)

# exp
lmbd_z_exp = 1.5
ts_exp = 2

lmbd_s_exp = 1 / ts_exp

In [125]:
T_req = generate_request_time_linear(tz_min, tz_max, work_time)
T_processing = generate_process_time_linear(ts_min, ts_max, len(T_req))

processed_requests, fot_times = simulate(T_req, T_processing, work_time)

p0 = fot_times[0] / work_time
p1 = fot_times[1] / work_time
p2 = fot_times[2] / work_time

Q = processed_requests / len(T_req)
S = processed_requests / work_time
k = (fot_times[1] * 1 + fot_times[2] * 2) / work_time

print(p0)
print(p1)
print(p2)
print('========')
print(Q)
print(S)
print(k)


0.013211557794562718
0.2073524075313065
0.7794360346741308
0.3919393603253836
0.5888888888888889
1.7662244768795679


In [116]:
# Linear (test)
test_p = lmbd_z_lin / lmbd_s_lin

p0 = 1 / (1 + test_p + test_p**2 / 2)
p1 = test_p * p0
p2 = test_p**2 / 2 * p0

Q = 1 - p2
S = lmbd_z_lin * Q
k = test_p * Q

print(p0)
print(p1)
print(p2)
print('========')
print(Q)
print(S)
print(k)

0.064
0.28800000000000003
0.648
0.352
0.5279999999999999
1.5839999999999999


In [128]:
T_req = generate_request_time_exp(lmbd_z_exp, work_time)
T_processing = generate_process_time_exp(lmbd_s_exp, len(T_req))

processed_requests, fot_times = simulate(T_req, T_processing, work_time)

p0 = fot_times[0] / work_time
p1 = fot_times[1] / work_time
p2 = fot_times[2] / work_time

Q = processed_requests / len(T_req)
S = processed_requests / work_time
k = (fot_times[1] * 1 + fot_times[2] * 2) / work_time

print(p0)
print(p1)
print(p2)
print('========')
print(Q)
print(S)
print(k)


0.1188557496769138
0.3571851718032642
0.523959078519822
0.4787037037037037
0.7180555555555556
1.4051033288429082


In [127]:
# Exp (test)
test_p = lmbd_z_exp / lmbd_s_exp

p0 = 1 / (1 + test_p + test_p**2 / 2)
p1 = test_p * p0
p2 = test_p**2 / 2 * p0

Q = 1 - p2
S = lmbd_z_exp * Q
k = test_p * Q

print(p0)
print(p1)
print(p2)
print('========')
print(Q)
print(S)
print(k)

0.11764705882352941
0.3529411764705882
0.5294117647058824
0.47058823529411764
0.7058823529411764
1.4117647058823528


In [137]:
table_cols = ['p0', 'p1', 'p2', 'Q', 'S', 'k']

In [172]:
rows = 5
table_vals = np.zeros((rows + 1, 6))

test_p = lmbd_z_lin / lmbd_s_lin
p0 = 1 / (1 + test_p + test_p**2 / 2)
p1 = test_p * p0
p2 = test_p**2 / 2 * p0
Q = 1 - p2
S = lmbd_z_lin * Q
k = test_p * Q
table_vals[0, :] = [p0, p1, p2, Q, S, k]

for i in range(5):
    T_req = generate_request_time_linear(tz_min, tz_max, work_time)
    T_processing = generate_process_time_linear(ts_min, ts_max, len(T_req))

    processed_requests, fot_times = simulate(T_req, T_processing, work_time)

    p0 = fot_times[0] / work_time
    p1 = fot_times[1] / work_time
    p2 = fot_times[2] / work_time

    Q = processed_requests / len(T_req)
    S = processed_requests / work_time
    k = (fot_times[1] * 1 + fot_times[2] * 2) / work_time

    table_vals[i + 1, :] = [p0, p1, p2, Q, S, k]

pd.DataFrame(table_vals, columns=table_cols)

,p0,p1,p2,Q,S,k
0,0.064000,0.288000,0.648000,0.352000,0.528000,1.584000
1,0.012135,0.213853,0.774012,0.390541,0.587222,1.761877
2,0.012708,0.211520,0.775772,0.392037,0.588056,1.763064
3,0.014541,0.213470,0.771989,0.388714,0.583611,1.757448
4,0.014679,0.211908,0.773413,0.391086,0.585000,1.758735
5,0.012708,0.208957,0.778335,0.392586,0.588333,1.765627


In [165]:
rows = 5
table_vals = np.zeros((rows + 1, 6))

test_p = lmbd_z_exp / lmbd_s_exp
p0 = 1 / (1 + test_p + test_p**2 / 2)
p1 = test_p * p0
p2 = test_p**2 / 2 * p0
Q = 1 - p2
S = lmbd_z_exp * Q
k = test_p * Q
table_vals[0, :] = [p0, p1, p2, Q, S, k]

for i in range(5):
    T_req = generate_request_time_exp(lmbd_z_exp, work_time)
    T_processing = generate_process_time_exp(lmbd_s_exp, len(T_req))

    processed_requests, fot_times = simulate(T_req, T_processing, work_time)

    p0 = fot_times[0] / work_time
    p1 = fot_times[1] / work_time
    p2 = fot_times[2] / work_time

    Q = processed_requests / len(T_req)
    S = processed_requests / work_time
    k = (fot_times[1] * 1 + fot_times[2] * 2) / work_time

    table_vals[i + 1, :] = [p0, p1, p2, Q, S, k]

pd.DataFrame(table_vals, columns=table_cols)

,p0,p1,p2,Q,S,k
0,0.117647,0.352941,0.529412,0.470588,0.705882,1.411765
1,0.112164,0.358013,0.529824,0.468581,0.698056,1.417660
2,0.125545,0.353669,0.520786,0.473774,0.697500,1.395241
3,0.115612,0.349441,0.534947,0.463919,0.703611,1.419335
4,0.127333,0.357617,0.515050,0.468089,0.694722,1.387718
5,0.117239,0.352498,0.530263,0.465209,0.709444,1.413024
